# Curricular Analytics - What if? Simulator

Hi! This notebook is here to walk you through simulating changes to a curriculum. You give us the file and the desired change and we'll tell you how that affects complexity and centrality, and why. There are animated examples for each section. If there's an error you don't understand, or you want to suggest improvements, please contact [a1amaya@ucsd.edu](mailto:a1amaya@ucsd.edu). But first, let us do some basic-set up. Run the cell below to do that: (if you've never used a Jupyter Notebook, press Shift+Enter to run a selected cell)

In [ ]:
using CurricularAnalytics, CurricularAnalyticsDiff

Ok! We're all set! There are four modifications you can make: you can add a course, remove a course, add a prereq and remove a prereq. After adding the curriculum you wish to change by running the next cell, just scroll on down to the section you want to work with. Currently, we'll be adding the FILENAME file in the targets folder. Feel free to upload your own curriculum file and change the file path.

<img src="./assets/SelectAFile.gif" />

In [ ]:
curr = read_csv("./targets/FILENAME.csv");
# dont' worry about this, this is just to make sure we're working with curricula, not degree plans
if typeof(curr) == DegreePlan
    curr = curr.curriculum
end

# Make modifications

## Add a Course:


Let's add a course. A course is defined by a few things. It has a name, credit_hours, and prerequisites. It also usually has a list of classes that depend on it. i.e. the classes that have it as a prerequisite. Let's define those things right now. 

NOTE:
When editing the prereqs field, make sure the "COURSE NAME #" field is replaced by the name of the existing course in the target curriculum that is to become a prereq of your new course. The same concept applies to the "COURSE NAME #" in the dependencies field. Once you're done adding prerequisites and dependencies, delete the extra fields.
Pre refers to the type of prerequisite. For now, let's leave that alone.

Note that in a curriculum where MATH 20B is called "MATH 20B/10B" or something like that you must use the name used in the curriculum file, otherwise this will NOT work. 

<img src="./assets/AddACourse.gif" />

In [ ]:
new_course_name = "COURSE NAME";
new_course_credit_hours = 4.0; # defualt, you can change it to 1.0,2.0,3.0, etc
prereqs = Dict("PREREQ 1" => pre,
               "PREREQ 2" => pre,
               "PREREQ 3" => pre);
dependencies = Dict("DEPENDENT 1" => pre,
                    "DEPENDENT 2" => pre,
                    "DEPENDENT 3" => pre);

Run the above cell once you're done, then run this one:

In [ ]:
new_curric = add_course(new_course_name, curr, new_course_credit_hours, prereqs, dependencies);
errors = IOBuffer();
isvalid_curriculum(new_curric,errors)

If you're getting an error, please make sure all the course names you're using match the ones in the file exactly.

## Remove a course

Removing a course is pretty easy. Just tell us the name of the course to be removed in the field below. Remember, it must match exactly with the name used in the curriculum file. 

<img src="./assets/RemoveACourse.gif" style="margin: -20px 0px -150px 0px"/>

In [ ]:
course_to_remove = "COURSE NAME";

Then, run this cell:

In [ ]:
new_curric = remove_course(course_to_remove, curr);
errors = IOBuffer();
isvalid_curriculum(new_curric,errors)

## Add a prerequisite

To add a prerequisite to a course tell us the name of the course, and the name of the prerequisite you wish to add to it. Remember, both names must already exist in the curriculum and must match exactly with your names. For example, if MATH 20B shows up as "MATH 20B/10B" in your curriculum, use that.

<img src="./assets/AddAPrereq.gif" />

In [ ]:
course_name = "COURSE TO GET A PREREQUISITE";
prerequisite = "PREREQ NAME";
req_type = pre;

Once you're ready, run this cell:

In [ ]:
new_curric = add_prereq(course_name, prerequisite, curr, req_type);
errors = IOBuffer();
isvalid_curriculum(new_curric,errors)

## Remove a prerequisite

Removing a prerequisite needs the name of the course in question, and the name of the course which we want to remove as its prerequisite. Remember the names need to match the ones used in the curriculum file exactly.

<img src="./assets/RemoveAPrereq.gif" />

In [ ]:
course_name = "COURSE TO LOSE A PREREQUISITE";
prerequisite = "PREREQUISITE TO BE LOST";

All good? Run the cell below

In [ ]:
new_curric = remove_prereq(course_name,prerequisite,curr);
errors = IOBuffer();
isvalid_curriculum(new_curric,errors)

# How did that affect the Curricular Metrics?

This is an easy question to answer. Just run the cell below after having made a new_curric with one of the previous modifications

In [ ]:
results = curricular_diff(curr,new_curric);

Once you're done with that, choose how you want to see your results: executive summary or pretty print

In [ ]:
executive_summary_curriculum(results)

In [ ]:
pretty_print_curriculum_results(results, ALL)

# Wrapping up

You can play around with this file, and you can write your new curriculum to file with the following cell. Unfortunately, we can't visualize the curricula here in this notebook, but you can take them over to the curricular analytics website to view.

<img src="./assets/DownloadAFile.gif" />

In [ ]:
write_csv(new_curric, "./newfile.csv")

The full package is available to you now to use. If you wish to make more complex modifications, be our guest! With these very basic building blocks you can make very complex edits to curricula.

# Institutional What if:

This is slightly more experimental. The aim here is to ask the same questions, but at an institutional level. For example, if there is suddenly a new requirement in the calculus series, the math department may have inadvertently affected lots of people who aren't math majors. This process uses a custom-made script that generates a curriculum file that condenses all the curricula in a given year. It uses the canonical name field to encode which plans a given course appears in. In the case that both classes in a given prerequisite relationship have the same plan code in their canonical name we can infer that that specific prerequisite relationship exists in that plan. If you don't have such a custom file file, feel free to contact us or use the provided UCSD-based sample file, $\verb|condensed.csv|$ to play around with.

## Add a course

Like before, fill in the information and then run the cell. Any major that has a class that will depend on this new class is going to be affected. 

<img src="./assets/AddACourseInstitutional.gif" />

In [ ]:
UCSD = read_csv("./targets/condensed.csv")
new_course_name = "NEW COURSE NAME";
new_course_credit_hours = 4.0; # defualt, you can change it to 1.0,2.0,3.0, etc
prereqs = Dict("PREREQ 1" => pre,
               "PREREQ 2" => pre,
               "PREREQ 3" => pre);
dependencies = Dict("DEPENDENT 1" => pre,
                    "DEPENDENT 2" => pre,
                    "DEPENDENT 3" => pre);
affected = add_course_institutional(new_course_name, UCSD, new_course_credit_hours, prereqs,dependencies);

## Remove a course

Just tell us what course you want to remove. This one is very easy to answer. Every plan that course is in will be affected.

In [ ]:
UCSD = read_csv("./targets/condensed.csv");
course_to_remove_name = "NAME";
affected = delete_course_institutional(course_to_remove_name, UCSD);

## Add a prerequisite

What course is going to get a prerequisite? Just fill that in and run the cell. In the current version of this system, we assume that we're adding an AND-type prerequisite (i.e. a required one, as opposed to a prereq that is equivalent to another exisiting prerequisite). Thus, every plan that the course we're adding a prerequisite to will be affected.

In [ ]:
UCSD = read_csv("./targets/condensed.csv");
affected = add_prereq_institutional(UCSD, "COURSE TO GET PREREQ", "NEW PREREQ");

## Remove a prerequisite

Just tell us what prerequisite you want to remove from which course, and run the cell. We'll handle finding them. Every plan the changing course appears in will be affected.

In [ ]:
UCSD = read_csv("./targets/condensed.csv");
affected = delete_prerequisite_institutional("COURSE TO LOSE PREREQ", "PREREQ TO LOSE", UCSD);